In [4]:
search_for = 123
start_from = 0
threads = 10
thread = 3
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004036664962768555
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 350465005
type: [1, 1, 1, 1, 123] 123
cases of this type: 228886641
100000 17.183569694877058
200000 26.664591555266846
300000 26.648488586217727
400000 26.85021840678031
500000 29.38371287329384
600000 25.76597199178382
700000 25.545491502814503
800000 25.85354325223199
900000 24.73684137894719
1000000 11.1348557930651
1100000 25.165846653922895
1200000 25.613918351112485
1300000 25.09124335755584
1400000 25.88787110282675
1500000 25.77710288270609
1600000 24.93449311961365
1700000 25.262559921894393
1800000 25.05332785718783
1900000 10.22742921404448
2000000 25.668623971326063
2100000 30.164258558662837
2200000 29.098554926857922
2300000 29.3759574464426
2400000 30.070456745175505
2500000 28.174288336845585
2600000 27.863359307058076
2700000 27.528640041638404
2800000 17.563818533932835
2900000 22.19545627869889
3000000 27.789

29400000 24.031732510708423
29500000 24.08156059568597
29600000 12.418621213974005
29700000 23.755474838962353
29800000 24.1602131021578
29900000 24.251347234942923
30000000 22.276651044490944
30100000 13.149942655912769
30200000 24.621306319146576
30300000 23.664974534967904
30400000 23.54145596625065
30500000 22.90832000388317
30600000 21.390656824629772
30700000 16.37121279583323
30800000 15.255679557077636
30900000 21.44545258394574
31000000 22.26803890095404
31100000 23.35327203807785
31200000 23.611154463019272
31300000 24.69131000054553
31400000 15.699470970903775
31500000 21.126028978107954
31600000 24.145603048562318
31700000 24.433656498439227
31800000 24.12054118566315
31900000 17.762339340490676
32000000 18.01612576127548
32100000 24.54749711393806
32200000 22.957660652334077
32300000 23.730935535438924
32400000 22.06951946037872
32500000 21.11760915404266
32600000 8.777706166375372
32700000 22.213986945158847
32800000 21.024827989163697
32900000 22.8431635210582
33000000 2

59200000 20.28123840216843
59300000 20.498486555648775
59400000 21.14783730690684
59500000 21.299113558980377
59600000 22.511025130454325
59700000 21.925535446100156
59800000 21.244754963257122
59900000 20.55838087396037
60000000 21.335261080781418
60100000 9.193237727478028
60200000 19.568715411672468
60300000 18.83534683606575
60400000 19.619572368399204
60500000 9.800466689045118
60600000 17.414798597651213
60700000 20.012853603101025
60800000 19.22801946528997
60900000 16.84167266212451
61000000 13.937070645499807
61100000 20.156482291011375
61200000 21.35805005522568
61300000 20.89696022151196
61400000 21.306300140343822
61500000 22.00536587548656
61600000 21.142457573094088
61700000 22.879598884106354
61800000 19.629183458465235
61900000 17.301553241078583
62000000 13.242874204360373
62100000 18.2059206003336
62200000 19.716290138552964
62300000 19.38767887784067
62400000 7.665392880432029
62500000 19.103809387508868
62600000 18.7877615461885
62700000 18.8511485688389
62800000 10

89000000 16.821450452825
89100000 17.099856668918456
89200000 18.156383066643
89300000 18.67992043508978
89400000 18.913967709560286
89500000 18.6298778610789
89600000 17.790480955950372
89700000 16.841587015653793
89800000 16.889828710322263
89900000 17.660009904918006
90000000 16.376866729341685
90100000 8.19003442671871
90200000 16.575925879593147
90300000 6.526949251957082
90400000 17.346928552170823
90500000 8.75075966389265
90600000 17.102668967529503
90700000 17.075423376882057
90800000 17.019046097234174
90900000 16.984751344919907
91000000 17.378005210481824
91100000 18.571019454820963
91200000 18.59353082318442
91300000 19.01250629235252
91400000 17.915255838216385
91500000 16.75704127997631
91600000 16.546571536552587
91700000 16.906841635118113
91800000 16.88995759613857
91900000 15.944843892897367
92000000 8.384041968569067
92100000 13.416828761194443
92200000 10.14962484107714
92300000 11.376538472688305
92400000 13.606618179933587
92500000 16.64742134969611
92600000 16.5

118100000 1.7534040946958263
118200000 1.7568799453623956
118300000 1.7599785242163737
118400000 1.7693317551953858
118500000 1.788807337148438
118600000 2.0631011672758754
118700000 1.7805974946704044
118800000 1.7854384600624642
118900000 1.7905788480430842
119000000 1.8701729376977416
119100000 2.6111236377520233
119200000 23.232902812135396
119300000 21.899579151601
119400000 21.90912531647553
119500000 21.2401482775937
119600000 21.15779089151625
119700000 20.771880358425364
119800000 20.350002358448357
119900000 20.657144533911605
120000000 10.88905266084064
120100000 2.7276685226320034
120200000 19.410641052653904
120300000 20.334891306141373
120400000 20.806301322095006
120500000 20.497126222176885
120600000 21.67307471171433
120700000 21.232157472781417
120800000 21.60082497843839
120900000 22.180804933136884
121000000 21.76631987939277
121100000 17.905604837247694
121200000 17.70223987710719
121300000 16.827191657635858
121400000 17.29991036699697
121500000 17.273903222411047

...

233200000 2.2143382111578287
233300000 2.0534524318000114
233400000 2.3687213791068364
233500000 0.9933281231233875
233600000 0.5276191268698408
233700000 1.5306770742104683
233800000 2.0749320978722077
233900000 2.433695633902785
234000000 2.146084963873476
234100000 2.3382160578258637
234200000 2.0934185840140183
234300000 2.1218600207289295
234400000 1.9774736966953477
234500000 2.3858916012878515
234600000 2.3701888130679163
234700000 1.9041785205226343
234800000 2.1268643283551336
234900000 2.4894972938341495
235000000 2.4384336951695684
235100000 2.055993791102628
235200000 2.1393518821136785
235300000 1.9353944083230694
235400000 0.5350624163793126
235500000 0.5358351881393293
235600000 2.161122407462448
235700000 2.161510953014894
235800000 2.2069678406488067
235900000 2.2867901999968585
236000000 2.054090987563994
236100000 2.3087177622774044
236200000 1.9955070189818143
236300000 2.2013598620899684
236400000 2.418122723848273
236500000 2.2611242014638977
236600000 3.079451640

261900000 3.60760448016471
262000000 3.1307436777040363
262100000 3.247967671990739
262200000 3.0466589126361545
262300000 3.4288731663401424
262400000 4.025510698884084
262500000 3.73863836102687
262600000 3.295849826619953
262700000 3.3168137764764727
262800000 3.4556596343039576
262900000 3.245646725833863
263000000 3.215591120055182
263100000 2.949825882900152
263200000 3.9263275957418284
263300000 0.7341579827094575
263400000 0.733407862478935
263500000 2.746256698467662
263600000 3.167638125598908
263700000 3.189449151311242
263800000 3.8737452724231485
263900000 3.134377901805146
264000000 2.8889762663727403
264100000 3.3197095831686854
264200000 3.5353297484717467
264300000 4.460072137786242
264400000 3.998522507718007
264500000 3.539718380100489
264600000 3.35522156595552
264700000 3.6273550265033245
264800000 3.706592958735721
264900000 3.4724836842360163
265000000 3.15901784033516
265100000 3.088288405465166
265200000 0.7092331007909047
265300000 1.2609845136068563
265400000

290700000 1.9398936993479032
290800000 2.12490698457849
290900000 2.044651262283673
291000000 2.0203300734847716
291100000 2.0821698893772562
291200000 0.8025010598535769
291300000 0.49245336906412246
291400000 1.8155398934179545
291500000 1.9137369605339867
291600000 2.0159862585901323
291700000 2.0577083089060486
291800000 2.1039507939999136
291900000 1.8946087705680397
292000000 2.065867516144226
292100000 2.182101884499934
292200000 2.434884175176481
292300000 2.001963518972158
292400000 2.104320015694426
292500000 1.827992976026366
292600000 1.9455133971038758
292700000 2.029811994682779
292800000 2.1102489577616255
292900000 1.901551830604156
293000000 1.7219062606453166
293100000 0.4731027959317333
293200000 0.47440938779416686
293300000 2.3160365597191985
293400000 1.9035032951501043
293500000 2.016514691395819
293600000 1.9832217801972891
293700000 2.1012517908225754
293800000 1.8304694426338672
293900000 2.1449871051622993
294000000 2.1202949124227297
294100000 2.305711286532

318500000 1.1664441423978973
318600000 1.3485258234640094
318700000 1.1545766202072203
318800000 1.2019935081972017
318900000 1.2101610357015928
319000000 1.2211165265182753
319100000 1.136381338373747
319200000 1.3893751593865984
319300000 1.263673382445097
319400000 1.2223742994661961
319500000 1.1430192955273952
319600000 1.091385612013489
319700000 1.339262074960977
319800000 1.3130353025895192
319900000 1.1413047543329695
320000000 1.1658446942376925
320100000 1.1722045158306196
320200000 1.1004312596041161
320300000 1.1856370450246303
320400000 1.177813469327549
320500000 1.1989372759195267
320600000 1.3161817672916218
320700000 1.073122334849308
320800000 1.085489821541498
320900000 1.09440905456034
321000000 1.2378988572649425
321100000 1.1446635080530645
321200000 1.1571352381912965
321300000 1.0619567027069032
321400000 1.1037430069622098
321500000 0.9956398965611589
321600000 1.0755397717133792
321700000 1.0408962539213698
321800000 1.0365414880433248
321900000 0.99057391989

345500000 0.20301041110093065
345600000 0.1545487021181716
345700000 0.18260569415706396
345800000 0.15799344817023475
345900000 0.15212491248641413
346000000 0.16110626991583904
346100000 0.13860533813826906
346200000 0.16253062055867912
346300000 0.11232123834918935
346400000 0.15834618435243764
346500000 0.1320716280122333
346600000 0.1347213718853891
346700000 0.1417272332757314
346800000 0.10653381600308419
346900000 0.1285389821012219
347000000 0.11395140443491272
347100000 0.12874282580828997
347200000 0.12630103996650377
347300000 0.12157093381760187
best so far: 0
type: [1, 3, 1, 41, 1] 123
cases of this type: 206763
347400000 0.07540053533664014
347500000 0.01461474176797271
best so far: 0
type: [1, 3, 41, 1, 1] 123
cases of this type: 5043
best so far: 0
type: [1, 41, 1, 1, 3] 123
cases of this type: 3321
best so far: 0
type: [1, 41, 1, 3, 1] 123
cases of this type: 1107
best so far: 0
type: [1, 41, 3, 1, 1] 123
cases of this type: 369
best so far: 0
type: [1, 123, 1, 1, 1] 